In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# --- CSVファイルの取得 ---
script_dir = os.path.dirname(__file__)
csv_files = glob.glob(os.path.join(script_dir, "*.csv"))

if not csv_files:
    raise FileNotFoundError("同じフォルダにCSVファイルが見つかりません。")

# --- カラーバー範囲の設定 ---
color_ranges = {
    "SiO2 Etching depth": (0, 1700),
    "SiO2 Etching rate": (0, 600),
    "Mask Etching depth": (0, 1400),
    "Mask Selectivity ratio": (1, 6),
    "Vpp": (0, 1000),
    "Vdc": (-800, 0)
}

for file_path in csv_files:
    print(f"読み込み中: {file_path}")

    # Shift-JIS で読み込み（Excel 日本語CSV対応）
    df = pd.read_csv(file_path, encoding="shift_jis", header=None)

    # --- 対象値名 ---
    full_name = str(df.iloc[0,0])
    if "/" in full_name:
        target_name = full_name.split("/", 1)[1].strip()
    else:
        target_name = full_name.strip()
    print(f"対象値: {target_name}")

    # --- Coil, Platen, z の抽出 ---
    coil_x = df.iloc[1,1:].astype(float).values
    platen_y = df.iloc[3:,0].astype(float).values
    z_values = df.iloc[3:,1:].astype(float).values

    x_list, y_list, z_list = [], [], []
    for i, y_val in enumerate(platen_y):
        for j, x_val in enumerate(coil_x):
            x_list.append(x_val)
            y_list.append(y_val)
            z_list.append(z_values[i, j])
    x = np.array(x_list)
    y = np.array(y_list)
    z = np.array(z_list)

    # --- 多項式回帰 ---
    X = np.column_stack((x, y))
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)
    model = LinearRegression()
    model.fit(X_poly, z)

    # --- グリッド作成 ---
    xi = np.linspace(0, 900, 100)
    yi = np.linspace(0, 250, 50)
    Xi, Yi = np.meshgrid(xi, yi)
    XY_poly = poly.transform(np.column_stack((Xi.ravel(), Yi.ravel())))
    Zi = model.predict(XY_poly).reshape(Xi.shape)

    # --- 下限値を0に制限（Vdcは除外） ---
    if target_name != "Vdc":
        Zi = np.clip(Zi, 0, None)

    # --- vmin, vmax を設定 ---
    vmin, vmax = color_ranges.get(target_name, (None, None))

    # --- 回帰式作成（係数5桁表示） ---
    coef = model.coef_
    intercept = model.intercept_
    feature_names = poly.get_feature_names_out(['X','Y'])
    equation = rf"$Z = {intercept:.5f}"
    for i, c in enumerate(coef):
        if c >= 0:
            sign = " + "
        else:
            sign = " - "
            c = abs(c)
        term = feature_names[i].replace(" "," \\cdot ").replace("^2","^{2}")
        equation += f"{sign}{c:.5f} {term}"
    equation += "$"

    from mpl_toolkits.mplot3d import Axes3D  # 3Dプロット用
    from matplotlib import cm  # カラーマップ用

    # --- 3Dプロット描画 ---
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # サーフェスプロット
    surf = ax.plot_surface(Xi, Yi, Zi,
                           cmap=cm.viridis,  # 色は残す場合
                           edgecolor='none',
                           alpha=0.9)

    # 元の測定点も散布図で追加
    ax.scatter(x, y, z, c='red', s=50, edgecolors="black", label="Measured")

    # 軸ラベル
    ax.set_xlabel("Coil [W]")
    ax.set_ylabel("Platen [W]")
    ax.set_zlabel(target_name)

    # タイトル
    ax.set_title(f"3D Mapping of {target_name}")

    # カラーバーを付ける（高さ方向の値を色で補助的に表示）
    fig.colorbar(surf, ax=ax, shrink=0.5, aspect=10, label=target_name)

    plt.legend()
    plt.show()

    # 学習データに対するR²
    r2 = model.score(X_poly, z)
    print(f"{target_name} の決定係数 R² = {r2:.3f}")

    # ラベル・タイトル
    plt.xlabel("Coil [W]")
    plt.ylabel("Platen [W]")
    plt.title(f"Predicted {target_name} (Polynomial Fit)")
    plt.text(0.05, 0.95, equation, transform=plt.gca().transAxes,
             fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.7))
    plt.text(0.05, 0.9, f"R² = {r2:.3f}", transform=plt.gca().transAxes,
         fontsize=12, verticalalignment='top',
         bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.7))

    # --- ファイル保存 ---
    safe_name = re.sub(r'[\\/:*?"<>|]', '_', str(target_name))
    output_file = f"{safe_name}_prediction.png"
    output_path = os.path.join(script_dir, output_file)
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"グラフを保存しました: {output_path}")